# Extract table from html file

In [172]:
import re

import numpy as np
import pandas as pd

In [318]:
# Settings

symbols = [
    'AAPL', 'AMZN', 'GOOG', 'JNJ', 'JPM', 'KO', 'META', 
    'NVDA', 'PYPL', 'TSLA', 'UNH', 'V', 'XOM'
    ]

In [322]:
# Functions
def read_html(sym):
    file = "C:/Users\phili\OneDrive\GitHub\Portfolio_Management\macrotrends\{0}_income.htm".format(sym)
    HtmlFile = open(file, 'r', encoding='utf-8')
    source_code = HtmlFile.read() 

    return source_code

def str_col_cleaner(x):
    # Catch empty records
    if x == '""':
        return 0
    else:
        return float(eval(x))

def extract_table_from_html(source_code):

    # Extract table
    t = re.findall(r"var originalData = .*", source_code)

    titles = re.findall(r"field_name.*?('>.*?)popup_icon", t[0], re.S)
    values = re.findall(r'\",?("2022.*?)}', t[0], re.S)

    # Clean titles
    titles = [re.findall(r">([A-Za-z &-\\]+)<", i) for i in titles]
    # Replace \\ /
    titles = [i[0].replace("\\", "") for i in titles]

    df = pd.DataFrame()

    for idx, title in enumerate(titles):

        temp = [i.split(":") for i in values[idx].split(",")]
        temp = pd.DataFrame.from_dict({i:[j] for i,j in temp}, orient='index', columns=[title]).reset_index()
        # There are some weird quotation marks. cannot replace them easily
        temp[title] = temp[title].apply(lambda x: str_col_cleaner(x))
        temp['index'] = [eval(i) for i in temp['index']]
        if idx == 0:
            df = df.append(temp)
        else:
            df = df.join(temp.set_index('index'), on='index')

    return df
    
    

In [324]:
for sym in symbols:
    print(sym)

    source_code = read_html(sym)

    df = extract_table_from_html(source_code)

    df.to_csv("C:/Users\phili\OneDrive\GitHub\Portfolio_Management\macrotrends\{0}_income.csv".format(sym), sep=";", decimal=",")


AAPL
AMZN
GOOG
JNJ
JPM
KO
META
NVDA
PYPL
TSLA
UNH
V
XOM
